[![](https://mermaid.ink/img/pako:eNo1jkELgkAUhP_K450M9A94CEohgg5SQlDb4eGuKemurG-pUP97a-JtZr5hmAELIxXGWDbmXVRkGU5noXf3kuKSItcrCzsna_OAKNqO16ruuznKjiPsg1x9eCP0fmYwJhXxIcsXmASrNY47x0A98FJP_lN5fhkhvS_jGGKrbEu19FcGoQEEcqVaJTD2UpJ9CRR68j1ybC5fXWDM1qkQXSeJVVrT01KL_nTT-7QjfTNm9dMPgGJNAQ?type=png)](https://mermaid.live/edit#pako:eNo1jkELgkAUhP_K450M9A94CEohgg5SQlDb4eGuKemurG-pUP97a-JtZr5hmAELIxXGWDbmXVRkGU5noXf3kuKSItcrCzsna_OAKNqO16ruuznKjiPsg1x9eCP0fmYwJhXxIcsXmASrNY47x0A98FJP_lN5fhkhvS_jGGKrbEu19FcGoQEEcqVaJTD2UpJ9CRR68j1ybC5fXWDM1qkQXSeJVVrT01KL_nTT-7QjfTNm9dMPgGJNAQ)

# **Installing all necessary libraries**

In [ ]:
!pip install -U openai-whisper
!pip install gradio
!pip install openai
!pip install TTS

# **Importing all necessary libraries**

In [ ]:
import whisper
import gradio as gr
import openai
from TTS.api import TTS

import warnings
warnings.filterwarnings('ignore')

## **Text to Speech (TTS) part**

In [ ]:
TTS.list_models()

In [ ]:
# lets take one random model (en)
model_name = TTS.list_models()[9]
tts = TTS(model_name)

## **Whisper part**

In [ ]:
whisper.available_models()

In [ ]:
# model can be passed as dropdown in gradio UI
model = whisper.load_model("tiny.en")
model.device

## **CHAT GPT PART**


In [9]:
import json
with open('env_vars.json', 'r') as f:
    env_vars = json.load(f)

openai.api_key = env_vars["OPENAI_API_KEY"]

In [10]:
messages = [
    {"role": "system", "content": "You are a helpful and kind AI Assistant."},
]

def chatgpt_api(input):
    if input:
        messages.append({"role": "user", "content": input})
        chat = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=messages
        )
        reply = chat.choices[0].message.content
        messages.append({"role": "assistant", "content": reply})
        return reply

## **Main function to transcribe**

In [11]:
def transcribe(audio):
    audio_to_text = model.transcribe(audio)["text"]

    text_to_audio = chatgpt_api(audio_to_text)

    tts.tts_to_file(text=text_to_audio, file_path="output.wav")

    return (audio_to_text, text_to_audio, "output.wav")

## **Gradio UI**

In [ ]:
output_1 = gr.Textbox(label="YOUR QUESTION: ")
output_2 = gr.Textbox(label="ANSWER(TEXT): ")
output_3 = gr.Audio(label="ANSWER(AUDIO): ", upload="output.wav")

gr.Interface(
    title = 'ASK ME ANYTHING!',
    fn=transcribe,
    inputs=[
            gr.Audio(source="microphone", type="filepath"),
            ],

    outputs=[
        output_1,  output_2, output_3
    ]).launch(share=True)